In [1]:
import os, random, shutil
import numpy as np
import pandas as pd
import PIL
#import keras
import itertools
from PIL import Image

import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
#from skimage import feature, data, io, measure
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
%matplotlib inline 

import cv2
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.layers import BatchNormalization
#from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.callbacks import TensorBoard
import io
import time
import datetime
from tensorflow.keras.optimizers import Adam,Adadelta
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
batch_size_train = 25
num_classes= 3
IMAGE_SIZE=[200,200]
classes_required = ['child','young','old']

In [3]:
datagen = ImageDataGenerator(rescale=1.0/255.0)

train_path = '/home/dell/Desktop/datasets/AGEGENDER/agedb/train'
train_batches = datagen.flow_from_directory(train_path, target_size=(200,200), classes=classes_required, batch_size=batch_size_train)


Found 37646 images belonging to 3 classes.


In [4]:
model=Sequential()
model.add(Conv2D(16,kernel_size=(3,3), activation="relu" ,input_shape=IMAGE_SIZE + [3], padding='same'))

model.add(Conv2D(32, kernel_size=(3,3), activation="relu",padding='same'))
model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.30))

model.add(Conv2D(64, kernel_size=(3,3), activation="relu",padding='same'))
#model.add(BatchNormalization()) 
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.35))

model.add(Conv2D(128, kernel_size=(3,3), activation="relu",padding='same'))
#model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization()) 
model.add(Dropout(0.45))

model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(num_classes, activation="softmax"))


In [ ]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(train_batches))

In [8]:
import numpy as np
import math

# labels_dict : {ind_label: count_label}
# mu : parameter to tune 

def create_class_weight(labels_dict,mu=0.15):
    total = np.sum(list(labels_dict.values()))
    keys = labels_dict.keys()
    class_weight = dict()
    
    for key in keys:
        score = math.log(mu*total/float(labels_dict[key]))
        class_weight[key] = score if score > 1.0 else 1.0
    
    return class_weight

# random labels_dict
labels_dict = {0: 12114, 1: 18990, 2: 6542}

create_class_weight(labels_dict)

{0: 1.0, 1: 1.0, 2: 1.0}

In [ ]:
class_weight = {0: 50.,
                1: 50.,
                2: 1.}

In [9]:
opt = Adadelta(learning_rate=0.0001, decay=1e-6)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model.fit_generator(train_batches, steps_per_epoch=50, epochs=100, verbose=1)

/home/dell/.local/lib/python3.6/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/100
50/50 [==============================] - 10s 132ms/step - loss: 2.1028 - accuracy: 0.2704
Epoch 2/100
50/50 [==============================] - 7s 137ms/step - loss: 1.8751 - accuracy: 0.3144
Epoch 3/100
50/50 [==============================] - 7s 138ms/step - loss: 1.7921 - accuracy: 0.3216
Epoch 4/100
50/50 [==============================] - 7s 142ms/step - loss: 1.6304 - accuracy: 0.3448
Epoch 5/100
50/50 [==============================] - 7s 134ms/step - loss: 1.6039 - accuracy: 0.3536
Epoch 6/100
50/50 [==============================] - 7s 137ms/step - loss: 1.5089 - accuracy: 0.3704
Epoch 7/100
50/50 [==============================] - 8s 161ms/step - loss: 1.5054 - accuracy: 0.3892
Epoch 8/100
50/50 [==============================] - 8s 168ms/step - loss: 1.5518 - accuracy: 0.3736
Epoch 9/100
50/50 [==============================] - 10s 194ms/step - loss: 1.4327 - accuracy: 0.4032
Epoch 10/100
50/50 [==============================] - 10s 193ms/step - loss: 1.3884 - acc

50/50 [==============================] - 10s 208ms/step - loss: 1.1494 - accuracy: 0.5104
Epoch 81/100
50/50 [==============================] - 11s 208ms/step - loss: 1.1638 - accuracy: 0.5192
Epoch 82/100
50/50 [==============================] - 11s 211ms/step - loss: 1.2509 - accuracy: 0.4904
Epoch 83/100
50/50 [==============================] - 11s 210ms/step - loss: 1.1607 - accuracy: 0.5048
Epoch 84/100
50/50 [==============================] - 11s 208ms/step - loss: 1.1170 - accuracy: 0.5192
Epoch 85/100
50/50 [==============================] - 11s 208ms/step - loss: 1.2091 - accuracy: 0.5000
Epoch 86/100
50/50 [==============================] - 10s 207ms/step - loss: 1.1662 - accuracy: 0.5032
Epoch 87/100
50/50 [==============================] - 11s 208ms/step - loss: 1.1455 - accuracy: 0.5088
Epoch 88/100
50/50 [==============================] - 11s 211ms/step - loss: 1.1383 - accuracy: 0.5128
Epoch 89/100
50/50 [==============================] - 11s 208ms/step - loss: 1.2270 - 

In [19]:
model.save("model_age.h5")
model_loaded = load_model("model_age.h5")

In [18]:
from tensorflow.keras.models import load_model

In [22]:
test_image=cv2.imread("/home/dell/Desktop/datasets/AGEGENDER/agedb/val/child/18_0_1_20170113135429329.jpg.chip.jpg")
test_image = np.expand_dims(test_image, axis=0)
out=np.argmax(model_loaded.predict(test_image))
Ans=np.where(out==0, "Young",
         (np.where(out==1,"Old",
         "Child"))).item()
print(Ans+" age person.")

Old age person.
